The purpose of this notebook is to create example json files (similar to however a backend API would be serving data) that contain data from the individual tornado simulations. These can be used to drive histograms, etc on the next version of the IMPACTS dashboard.

In [1]:
import pathlib
import glob
import os

import argparse
import geojson
import pandas as pd
import json
import random
import numpy as np

from datetime import datetime, timedelta

In [2]:
jsonResponse = []

In [6]:
# Read in example psv
file = '../data/output/examples/20110427/psvs/20110427162927_schools_fema_time.psv.gz'
df = pd.read_csv(file, sep="|")

### Bin by rating

In [7]:
bin_labels = ['0-1','2-3','4-5']
# lower bound is not inclusive, so we shift the thresholds one to the left
cut_bins = [-1,1,3,5]

# Add the hour bins to the original dataframe
df['ratingbin'] = pd.cut(df['rating'], bins=cut_bins, labels=bin_labels)
df[['rating','ratingbin','time']]

,rating,ratingbin,time
0,3,2-3,13
1,2,2-3,20
2,4,4-5,16
3,0,0-1,9
4,0,0-1,11
...,...,...,...
956145,2,2-3,2
956146,0,0-1,21
956147,0,0-1,1
956148,4,4-5,11


In [8]:
weak = df[df['ratingbin'] == '0-1']
sig = df[df['ratingbin'] == '2-3']
vio = df[df['ratingbin'] == '4-5']

In [9]:
weakSum = weak.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]
sigSum = sig.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]
vioSum = vio.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]

weakResponse = {'Level':'0-1'}
sigResponse = {'Level':'2-3'}
vioResponse = {'Level':'4-5'}

weakResponse['Sims'] = weakSum.reset_index().values.tolist()
sigResponse['Sims'] = sigSum.reset_index().values.tolist()
vioResponse['Sims'] = vioSum.reset_index().values.tolist()

In [10]:
jsonResponse.append(weakResponse)
jsonResponse.append(sigResponse)
jsonResponse.append(vioResponse)

In [11]:
jsonResponse

[{'Level': '0-1',
  'Sims': [[1, 198064, 7, 1175, 65, 85],
   [2, 148768, 8, 875, 38, 57],
   [3, 294000, 3, 1025, 58, 81],
   [4, 174320, 3, 1400, 45, 47],
   [5, 53952, 1, 325, 25, 15],
   [6, 149760, 0, 975, 42, 39],
   [7, 167008, 6, 1025, 55, 60],
   [8, 387232, 8, 2050, 95, 156],
   [9, 135440, 2, 1400, 40, 47],
   [10, 284336, 7, 1200, 58, 100],
   [11, 254688, 4, 525, 36, 71],
   [12, 115920, 2, 225, 19, 45],
   [13, 72848, 2, 875, 19, 36],
   [14, 250496, 4, 1050, 46, 102],
   [15, 47872, 0, 525, 14, 27],
   [16, 129744, 1, 1125, 24, 48],
   [17, 96592, 1, 800, 32, 48],
   [18, 61584, 1, 700, 33, 41],
   [19, 38512, 1, 475, 31, 26],
   [20, 204560, 4, 950, 60, 53],
   [21, 107168, 2, 675, 60, 42],
   [22, 184608, 4, 1450, 56, 70],
   [23, 99200, 0, 825, 26, 26],
   [24, 104608, 0, 1325, 27, 36],
   [25, 138704, 1, 800, 37, 25],
   [26, 89344, 3, 375, 25, 27],
   [27, 136784, 6, 1200, 61, 67],
   [28, 94640, 3, 350, 33, 76],
   [29, 187248, 11, 1600, 78, 92],
   [30, 35424, 1, 

In [12]:
with open('../data/output/examples/20110427/processed/jsonResponse_sims_newnat.json','w') as fp:
    json.dump(jsonResponse, fp)

### Make an example file just for FEMA regions
Need to know how I'll be serving up data before going any farther
For the time being, just want an additional example such that I can code functionality in javascript 

In [3]:
jsonResponse = []

In [4]:
# Read in example psv
file = '../data/output/examples/20110427/psvs/20110427162927_schools_fema_time.psv.gz'
df = pd.read_csv(file, sep="|")

In [5]:
df.head()

,sim,population,distance,rating,states,counties,wfos,hospitals,hospitalbeds,mobileparks,mobilehomes,psubstations,plines,slon,slat,elon,elat,schools,fema,time
0,1,2544,4.4415,3,MS,28081,MEG,0,0,0,0,0,0,-88.6066,34.3619,-88.5379,34.3921,0,4,13
1,1,624,3.9692,2,KY,"21051,21121",JKL,0,0,0,0,0,0,-83.8111,37.0729,-83.7601,37.1134,2,4,20
2,1,3280,27.0073,4,"AL,GA","1019,13055,13115","BMX,FFC",0,0,0,0,3,3,-85.5258,34.2540,-85.1459,34.4873,1,"4,4",16
3,1,928,1.1927,0,AL,1089,HUN,0,0,0,0,0,0,-86.4543,35.0071,-86.4403,35.0201,0,4,9
4,1,6464,0.1907,0,PA,42049,CLE,0,0,0,0,0,0,-79.9165,42.1845,-79.9137,42.1863,0,3,11


In [14]:
femaBrokenOut = df.assign(category=df['fema'].str.split(',')).explode('category').reset_index(drop=True)

# Remove row if state is NaN
femaBrokenOut = femaBrokenOut[femaBrokenOut['category'].notna()]

# Grab a list of the unique states in the simulation
femaImpacted = femaBrokenOut['category'].unique().tolist()

In [46]:
# Just grab Region 6
df_6 = femaBrokenOut[femaBrokenOut['category'] == '6']
df_6.head()

,sim,population,distance,rating,states,counties,wfos,hospitals,hospitalbeds,mobileparks,...,psubstations,plines,time,slon,slat,elon,elat,schools,fema,category
2,1,0,4.5373,2,TX,48023,OUN,0,0,0,...,0,0,2,-99.0422,33.6459,-98.9941,33.6980,0,6,6
3,1,80,0.7748,0,TX,48143,FWD,0,0,0,...,0,0,22,-98.1495,32.0895,-98.1437,32.0995,0,6,6
4,1,368,3.8219,1,TX,"48367,48497",FWD,0,0,0,...,0,0,23,-97.8818,33.0339,-97.8283,33.0662,0,6,6
5,1,144,0.5527,0,TX,48035,FWD,0,0,0,...,0,0,1,-97.4614,32.2109,-97.4531,32.2147,0,6,6
7,1,0,0.5467,2,TX,48503,FWD,0,0,0,...,0,0,23,-98.7699,33.2014,-98.7633,33.2071,0,6,6


In [19]:
impacts = ['population','hospitals','schools','mobilehomes','psubstations']

for fema in femaImpacted:
    
    jsonResponse = []
    
    ind_fema = femaBrokenOut[femaBrokenOut['category'] == fema]
    
    weak = ind_fema[ind_fema['ratingbin'] == '0-1']
    sig = ind_fema[ind_fema['ratingbin'] == '2-3']
    vio = ind_fema[ind_fema['ratingbin'] == '4-5']
    
    weakSum = weak.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]
    sigSum = sig.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]
    vioSum = vio.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]

    weakResponse = {'Level':'0-1'}
    sigResponse = {'Level':'2-3'}
    vioResponse = {'Level':'4-5'}

    weakResponse['Sims'] = weakSum.reset_index().values.tolist()
    sigResponse['Sims'] = sigSum.reset_index().values.tolist()
    vioResponse['Sims'] = vioSum.reset_index().values.tolist()
    
    jsonResponse.append(weakResponse)
    jsonResponse.append(sigResponse)
    jsonResponse.append(vioResponse)
    
    with open(f'../data/output/examples/20110427/processed/jsonResponse_sims_{fema}.json','w') as fp:
        json.dump(jsonResponse, fp)

Manually writing out individual FEMA groups

In [48]:
weak = df_6[df_6['ratingbin'] == '0-1']
sig = df_6[df_6['ratingbin'] == '2-3']
vio = df_6[df_6['ratingbin'] == '4-5']

In [49]:
weakSum = weak.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]
sigSum = sig.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]
vioSum = vio.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]

weakResponse = {'Level':'0-1'}
sigResponse = {'Level':'2-3'}
vioResponse = {'Level':'4-5'}

weakResponse['Sims'] = weakSum.reset_index().values.tolist()
sigResponse['Sims'] = sigSum.reset_index().values.tolist()
vioResponse['Sims'] = vioSum.reset_index().values.tolist()

In [50]:
jsonResponse.append(weakResponse)
jsonResponse.append(sigResponse)
jsonResponse.append(vioResponse)

In [51]:
with open('../data/output/examples/jsonResponse_sims_6.json','w') as fp:
    json.dump(jsonResponse, fp)

### Make an example files for States
Need to know how I'll be serving up data before going any farther
For the time being, just want an additional example such that I can code functionality in javascript 

In [52]:
jsonResponse = []

In [53]:
# Read in example psv
file = '../data/output/examples/20210428124600_schools_fema.psv.gz'
df = pd.read_csv(file, sep="|")

In [54]:
df.head()

,sim,population,distance,rating,states,counties,wfos,hospitals,hospitalbeds,mobileparks,mobilehomes,psubstations,plines,time,slon,slat,elon,elat,schools,fema
0,1,1056,2.0662,1,NY,36013,BUF,0,0,0,0,0,0,22,-79.1592,42.4951,-79.1388,42.5209,4,2
1,1,480,2.6825,1,NY,36097,BGM,0,0,0,0,0,0,21,-76.9440,42.4153,-76.9021,42.4388,0,2
2,1,0,4.5373,2,TX,48023,OUN,0,0,0,0,0,0,2,-99.0422,33.6459,-98.9941,33.6980,0,6
3,1,80,0.7748,0,TX,48143,FWD,0,0,0,0,0,0,22,-98.1495,32.0895,-98.1437,32.0995,0,6
4,1,368,3.8219,1,TX,"48367,48497",FWD,0,0,0,0,0,0,23,-97.8818,33.0339,-97.8283,33.0662,0,6


In [20]:
stBrokenOut = df.assign(category=df['states'].str.split(',')).explode('category').reset_index(drop=True)

# Remove row if state is NaN
stBrokenOut = stBrokenOut[stBrokenOut['category'].notna()]

# Grab a list of the unique states in the simulation
statesImpacted = stBrokenOut['category'].unique().tolist()

In [22]:
impacts = ['population','hospitals','schools','mobilehomes','psubstations']

for state in statesImpacted:
    
    jsonResponse = []
    
    ind_state = stBrokenOut[stBrokenOut['category'] == state]
    
    weak = ind_state[ind_state['ratingbin'] == '0-1']
    sig = ind_state[ind_state['ratingbin'] == '2-3']
    vio = ind_state[ind_state['ratingbin'] == '4-5']
    
    weakSum = weak.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]
    sigSum = sig.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]
    vioSum = vio.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]

    weakResponse = {'Level':'0-1'}
    sigResponse = {'Level':'2-3'}
    vioResponse = {'Level':'4-5'}

    weakResponse['Sims'] = weakSum.reset_index().values.tolist()
    sigResponse['Sims'] = sigSum.reset_index().values.tolist()
    vioResponse['Sims'] = vioSum.reset_index().values.tolist()
    
    jsonResponse.append(weakResponse)
    jsonResponse.append(sigResponse)
    jsonResponse.append(vioResponse)
    
    with open(f'../data/output/examples/20110427/processed/jsonResponse_sims_{state}.json','w') as fp:
        json.dump(jsonResponse, fp)

Manually running states

In [56]:
# Just grab TX
df_TX = stBrokenOut[stBrokenOut['category'] == 'TX']
df_TX.head()

,sim,population,distance,rating,states,counties,wfos,hospitals,hospitalbeds,mobileparks,...,psubstations,plines,time,slon,slat,elon,elat,schools,fema,category
2,1,0,4.5373,2,TX,48023,OUN,0,0,0,...,0,0,2,-99.0422,33.6459,-98.9941,33.6980,0,6,TX
3,1,80,0.7748,0,TX,48143,FWD,0,0,0,...,0,0,22,-98.1495,32.0895,-98.1437,32.0995,0,6,TX
4,1,368,3.8219,1,TX,"48367,48497",FWD,0,0,0,...,0,0,23,-97.8818,33.0339,-97.8283,33.0662,0,6,TX
5,1,144,0.5527,0,TX,48035,FWD,0,0,0,...,0,0,1,-97.4614,32.2109,-97.4531,32.2147,0,6,TX
7,1,0,0.5467,2,TX,48503,FWD,0,0,0,...,0,0,23,-98.7699,33.2014,-98.7633,33.2071,0,6,TX


### Make Example Files for CWAs

In [23]:
cwaBrokenOut = df.assign(category=df['wfos'].str.split(',')).explode('category').reset_index(drop=True)

# Remove row if state is NaN
cwaBrokenOut = cwaBrokenOut[cwaBrokenOut['category'].notna()]

# Grab a list of the unique states in the simulation
cwasImpacted = cwaBrokenOut['category'].unique().tolist()

In [24]:
impacts = ['population','hospitals','schools','mobilehomes','psubstations']

for cwa in cwasImpacted:
    
    jsonResponse = []
    
    ind_cwa = cwaBrokenOut[cwaBrokenOut['category'] == cwa]
    
    weak = ind_cwa[ind_cwa['ratingbin'] == '0-1']
    sig = ind_cwa[ind_cwa['ratingbin'] == '2-3']
    vio = ind_cwa[ind_cwa['ratingbin'] == '4-5']
    
    weakSum = weak.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]
    sigSum = sig.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]
    vioSum = vio.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]

    weakResponse = {'Level':'0-1'}
    sigResponse = {'Level':'2-3'}
    vioResponse = {'Level':'4-5'}

    weakResponse['Sims'] = weakSum.reset_index().values.tolist()
    sigResponse['Sims'] = sigSum.reset_index().values.tolist()
    vioResponse['Sims'] = vioSum.reset_index().values.tolist()
    
    jsonResponse.append(weakResponse)
    jsonResponse.append(sigResponse)
    jsonResponse.append(vioResponse)
    
    with open(f'../data/output/examples/20110427/processed/jsonResponse_sims_{cwa}.json','w') as fp:
        json.dump(jsonResponse, fp)

### Bin By Rating

In [57]:
bin_labels = ['0-1','2-3','4-5']
# lower bound is not inclusive, so we shift the thresholds one to the left
cut_bins = [-1,1,3,5]

# Add the hour bins to the original dataframe
df_TX['ratingbin'] = pd.cut(df_TX['rating'], bins=cut_bins, labels=bin_labels)
df_TX[['rating','ratingbin','time']].head()

<ipython-input-57-691241fb6237>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_TX['ratingbin'] = pd.cut(df_TX['rating'], bins=cut_bins, labels=bin_labels)


,rating,ratingbin,time
2,2,2-3,2
3,0,0-1,22
4,1,0-1,23
5,0,0-1,1
7,2,2-3,23


In [58]:
weak = df_TX[df_TX['ratingbin'] == '0-1']
sig = df_TX[df_TX['ratingbin'] == '2-3']
vio = df_TX[df_TX['ratingbin'] == '4-5']

In [59]:
weakSum = weak.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]
sigSum = sig.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]
vioSum = vio.groupby('sim').sum().loc[:,('population','hospitals','mobilehomes','psubstations','schools')]

weakResponse = {'Level':'0-1'}
sigResponse = {'Level':'2-3'}
vioResponse = {'Level':'4-5'}

weakResponse['Sims'] = weakSum.reset_index().values.tolist()
sigResponse['Sims'] = sigSum.reset_index().values.tolist()
vioResponse['Sims'] = vioSum.reset_index().values.tolist()

In [60]:
jsonResponse.append(weakResponse)
jsonResponse.append(sigResponse)
jsonResponse.append(vioResponse)

In [61]:
with open('../data/output/examples/jsonResponse_sims_newTX.json','w') as fp:
    json.dump(jsonResponse, fp)

In [62]:
jsonResponse[0]

{'Level': '0-1',
 'Sims': [[1, 592, 0, 0, 0, 0],
  [2, 80, 0, 0, 4, 0],
  [3, 87136, 2, 0, 3, 19],
  [4, 304, 0, 0, 0, 2],
  [5, 368, 0, 0, 0, 0],
  [6, 1136, 0, 0, 0, 0],
  [8, 128, 0, 0, 0, 0],
  [9, 24976, 0, 600, 4, 20],
  [10, 29744, 2, 0, 3, 23],
  [11, 96, 0, 0, 0, 0],
  [12, 208, 0, 0, 0, 0],
  [13, 240, 0, 0, 0, 0],
  [14, 3824, 0, 0, 1, 3],
  [15, 464, 0, 0, 0, 0],
  [17, 1264, 0, 75, 0, 2],
  [18, 464, 0, 0, 0, 2],
  [19, 784, 0, 0, 0, 0],
  [20, 4528, 0, 0, 1, 2],
  [21, 64, 0, 0, 0, 0],
  [22, 91696, 1, 0, 12, 36],
  [23, 80, 0, 0, 0, 0],
  [24, 16, 0, 0, 0, 0],
  [25, 3376, 0, 0, 1, 1],
  [26, 1104, 0, 25, 0, 0],
  [27, 240, 0, 0, 0, 1],
  [28, 16, 0, 0, 1, 0],
  [30, 7952, 0, 0, 0, 3],
  [31, 240, 0, 0, 1, 3],
  [32, 16, 0, 0, 2, 3],
  [33, 688, 0, 0, 0, 2],
  [34, 48, 0, 0, 2, 3],
  [35, 29792, 3, 25, 1, 20],
  [36, 80, 0, 0, 0, 0],
  [37, 96, 0, 0, 0, 0],
  [38, 1472, 1, 0, 1, 8],
  [39, 112, 0, 0, 0, 0],
  [40, 3600, 0, 25, 5, 18],
  [42, 2144, 0, 175, 3, 0],
  [43, 8